<a href="https://colab.research.google.com/github/nitisha-pradhan/RealTime_Drowsiness_Detection_System/blob/main/Drowsiness_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing data from Kaggle#

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
 
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d akshitmadan/eyes-open-or-closed

 40% 9.00M/22.6M [00:01<00:01, 8.03MB/s]
100% 22.6M/22.6M [00:01<00:00, 15.2MB/s]


In [ ]:
!unzip \*.zip  && rm *.zip

#Data Generator and Augmentation#

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        shear_range=0.3,
        zoom_range=0.3
        )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/dataset/train',
        target_size=(48, 48),
        batch_size=32,
        class_mode='categorical',
        color_mode = 'grayscale'
        )

validation_generator = test_datagen.flow_from_directory(
        '/content/dataset/test',
        target_size=(48, 48),
        batch_size=32,
        class_mode='categorical',
        shuffle = False,
        color_mode = 'grayscale'
        )

Found 3400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


#Building model#

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
import warnings
warnings.filterwarnings('ignore')

In [ ]:
model = Sequential() 

model.add(Conv2D(128, (3, 3), padding="same", activation="relu", input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(units=130, activation="relu"))

model.add(Dense(units=2, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 48, 48, 128)       1280      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 24, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        73792     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)             

In [ ]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

#Training model#

In [ ]:
history = model.fit_generator(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10
107/107 [==============================] - 5s 41ms/step - loss: 0.3558 - accuracy: 0.8447 - val_loss: 0.0656 - val_accuracy: 0.9817
Epoch 2/10
107/107 [==============================] - 4s 40ms/step - loss: 0.1048 - accuracy: 0.9665 - val_loss: 0.0149 - val_accuracy: 0.9967
Epoch 3/10
107/107 [==============================] - 4s 40ms/step - loss: 0.0754 - accuracy: 0.9741 - val_loss: 0.0105 - val_accuracy: 1.0000
Epoch 4/10
107/107 [==============================] - 4s 40ms/step - loss: 0.0495 - accuracy: 0.9832 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 5/10
107/107 [==============================] - 4s 40ms/step - loss: 0.0476 - accuracy: 0.9847 - val_loss: 0.0169 - val_accuracy: 1.0000
Epoch 6/10
107/107 [==============================] - 4s 39ms/step - loss: 0.0461 - accuracy: 0.9853 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 7/10
107/107 [==============================] - 4s 40ms/step - loss: 0.0360 - accuracy: 0.9874 - val_loss: 9.2847e-04 - val_accuracy: 1.

In [ ]:
train_generator.class_indices

{'Closed_Eyes': 0, 'Open_Eyes': 1}

#Saving and downloading model as h5 file#

In [ ]:
model.save('drowsiness.h5')